In [1]:
from dataprocessing.utils.normalization import get_stats
import torch
import numpy as np
import random
from train import train
from utils.visualization import save_plots
import torch_geometric.transforms as T
from torch_geometric.datasets import Planetoid
from torch_geometric.loader import DataLoader

In [2]:
class objectview(object):
    def __init__(self, d):
        self.__dict__ = d

for args in [
        {'model_type': 'meshgraphnet',  
         'num_layers': 10,
         'batch_size': 1, 
         'hidden_dim': 10, 
         'num_nodes': None,
         'epochs': 100,
         'opt': 'adam', 
         'opt_scheduler': 'none', 
         'opt_restart': 0, 
         'weight_decay': 5e-4, 
         'lr': 0.001,
         'train_size': 45, 
         'test_size': 10, 
         'device':'cuda',
         'shuffle': True, 
         'save_velo_val': True,
         'save_best_model': True, 
         'checkpoint_dir': './best_models/',
         'postprocess_dir': './2d_loss_plots/'},
    ]:
        args = objectview(args)

#To ensure reproducibility the best we can, here we control the sources of
#randomness by seeding the various random number generators used in this Colab
#For more information, see: https://pytorch.org/docs/stable/notes/randomness.html
torch.manual_seed(5)  #Torch
random.seed(5)        #Python
np.random.seed(5)     #NumPy

In [3]:
if torch.backends.mps.is_available():
  device = 'mps'
elif torch.cuda.is_available():
  device = 'cuda'
else:
  device = 'cpu'
device = 'cpu'
print(device)
args.device = device

cpu


In [68]:
dataset = torch.load('data/trajectories/trajectory_1.pt')[:(args.train_size+args.test_size)]
data_stats = get_stats(dataset)
print(data_stats)
print(len(dataset))
print(dataset[0])

[tensor([3.8929e-01, 6.1321e-04, 8.7527e-01, 0.0000e+00, 0.0000e+00, 0.0000e+00,
        9.0618e-03, 9.0618e-03, 1.0661e-01, 0.0000e+00, 0.0000e+00]), tensor([2.4483e-01, 7.7642e-02, 3.3042e-01, 1.0000e-08, 1.0000e-08, 1.0000e-08,
        9.4761e-02, 9.4761e-02, 3.0862e-01, 1.0000e-08, 1.0000e-08]), tensor([-3.9781e-10,  1.1050e-11,  2.0158e-02]), tensor([0.0159, 0.0148, 0.0082]), tensor([-0.1814,  0.0014]), tensor([0.9233, 0.3153])]
55
Data(x=[1876, 11], edge_index=[2, 10788], edge_attr=[10788, 3], y=[1876, 2], p=[1876, 1], cells=[3518, 3], mesh_pos=[1876, 2])


In [5]:
import os
import collections
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import scipy.sparse as sp
import torch
from torch import Tensor
import torch_geometric
from torch_geometric.utils import to_networkx
from torch_geometric.datasets import Planetoid
import networkx as nx
from networkx.algorithms import community
import torch
import torch_scatter
import torch.nn as nn
from torch.nn import Linear, Sequential, LayerNorm, ReLU
from torch_geometric.nn.conv import MessagePassing
from dataprocessing.utils.normalization import normalize, get_stats
from torch_geometric.nn import GAE, VGAE, GCNConv
from torch_geometric.nn.pool import TopKPooling, global_max_pool
from torch_geometric.nn.unpool import knn_interpolate

In [75]:
input_dim_node, hidden_dim, latent_dim  = dataset[0].num_features, 32, 32
args.num_nodes = dataset[0].num_nodes
print(args.num_nodes)
model = GCNAutoEncoder(input_dim_node, hidden_dim, latent_dim, input_dim_node, args)
model = model.to(device)
print(model)
opt = torch.optim.Adam(model.parameters(), lr=0.01)


1876
GCNAutoEncoder(
  (node_encoder): GCNEncoder(
    (conv1): GCNConv(11, 32)
    (pool1): TopKPooling(32, ratio=0.2, multiplier=1.0)
    (conv2): GCNConv(32, 32)
    (pool2): TopKPooling(32, ratio=0.2, multiplier=1.0)
    (conv3): GCNConv(32, 64)
    (pool3): TopKPooling(64, ratio=0.1, multiplier=1.0)
    (pool4): TopKPooling(64, ratio=0.125, multiplier=1.0)
    (relu): ReLU()
  )
  (mlp): Sequential(
    (0): ReLU()
    (1): Linear(in_features=64, out_features=32, bias=True)
  )
  (node_decoder): GCNDecoder(
    (conv1): GCNConv(32, 32)
    (conv2): GCNConv(32, 11)
    (linear): Linear(in_features=32, out_features=1000, bias=True)
    (relu): ReLU()
  )
)


In [76]:
loader = DataLoader(dataset, batch_size=1)
sample = next(iter(loader))
print(sample)
test = model(sample.x.to(device), sample.edge_index.to(device))
print(test.shape)

DataBatch(x=[1876, 11], edge_index=[2, 10788], edge_attr=[10788, 3], y=[1876, 2], p=[1876, 1], cells=[3518, 3], mesh_pos=[1876, 2], batch=[1876], ptr=[2])
X : torch.Size([1876, 11])
Edge index:  torch.Size([2, 10788])
X post pool 1:  torch.Size([376, 32])
X post pool 2:  torch.Size([76, 32])
X post pool 3:  torch.Size([8, 64])
X post pool 4:  torch.Size([1, 64])
Encoded:  torch.Size([1, 64])
Post mlp:  torch.Size([1, 32])
torch.Size([1, 1000])


RuntimeError: mat1 and mat2 shapes cannot be multiplied (1x1000 and 32x32)

In [ ]:
from tqdm import trange
def train():
    losses = []
    test_losses = []
    best_test_loss = np.inf
    for epoch in trange(args.epochs, desc="Training", unit="Epochs"):
        total_loss = 0
        model.train()
        count = 0
        for __loader__, batch in enumerate(loader):
            #Note that normalization must be done before it's called. The unnormalized
            #data needs to be preserved in order to correctly calculate the loss
            batch=batch[0].to(device)
            opt.zero_grad()         #zero gradients each time
            z = model.encode(batch.x, batch.edge_index)
            loss = model.recon_loss(z, batch.pos_edge_label_index)
            loss = loss + (1 / batch.num_nodes) * model.kl_loss()
            loss.backward()         #backpropagate loss
            opt.step()
            total_loss += loss.item()
            count += 1
        total_loss /= count
        losses.append(total_loss)
        if epoch % (args.epochs/10) == 0:
            print("train loss", total_loss)

    return losses

@torch.no_grad()
def test(data):
    model.eval()
    z = model.encode(data.x, data.edge_index)
    return model.test(z, data.pos_edge_label_index, data.neg_edge_label_index)

In [ ]:
print(loss)

In [ ]:
G = to_networkx(data, to_undirected=True)
pos = nx.spring_layout(G, seed=42)
cent = nx.degree_centrality(G)
node_size = list(map(lambda x: x * 500, cent.values()))
cent_array = np.array(list(cent.values()))
threshold = sorted(cent_array, reverse=True)[10]
print("threshold", threshold)
cent_bin = np.where(cent_array >= threshold, 1, 0.1)
plt.figure(figsize=(12, 12))
nodes = nx.draw_networkx_nodes(G, pos, node_size=node_size,
                               cmap=plt.cm.plasma,
                               node_color=cent_bin,
                               nodelist=list(cent.keys()),
                               alpha=cent_bin)
edges = nx.draw_networkx_edges(G, pos, width=0.25, alpha=0.3)
plt.show()

In [ ]:
G = to_networkx(cylinder_data, to_undirected=True)
pos = nx.spring_layout(G, seed=42)
cent = nx.degree_centrality(G)
node_size = list(map(lambda x: x * 500, cent.values()))
cent_array = np.array(list(cent.values()))
threshold = sorted(cent_array, reverse=True)[10]
print("threshold", threshold)
cent_bin = np.where(cent_array >= threshold, 1, 0.1)
plt.figure(figsize=(12, 12))
nodes = nx.draw_networkx_nodes(G, pos, node_size=node_size,
                               cmap=plt.cm.plasma,
                               node_color=cent_bin,
                               nodelist=list(cent.keys()),
                               alpha=cent_bin)
edges = nx.draw_networkx_edges(G, pos, width=0.25, alpha=0.3)
plt.show()

In [ ]:
edge_index = cylinder_data.edge_index.numpy()
print(edge_index.shape)
edge_example = edge_index[:, np.where(edge_index[0] < 5)[0]]
edge_example

In [ ]:
node_example = np.unique(edge_example.flatten())
plt.figure(figsize=(10, 6))
G = nx.Graph()
G.add_nodes_from(node_example)
G.add_edges_from(list(zip(edge_example[0], edge_example[1])))
nx.draw_networkx(G, with_labels=False)

In [ ]:
print(G)
print(G.)

In [ ]:
# get edge and node attributes
edge_labels = nx.get_edge_attributes(G, seed=42)
node_states = nx.get_node_attributes(G, 'state')
pos = nx.spring_layout(G, seed=0)
# set node state positions
state_pos = {n: (x+0.12, y+0.05) for n, (x,y) in pos.items()}

# draw graph
nx.draw_networkx(G, pos, node_size=600)
# draw node state labels
nx.draw_networkx_labels(G, state_pos, labels=node_states, font_color='red')
# draw edge attributes
nx.draw_networkx_edge_labels(G, pos, edge_labels=edge_labels)